# Accessing info through SQLite DB

Queries and example functions for utilities

## Connect to the DB

In [1]:
# Create a new database
from pathlib import Path
# Path('my_data.db').touch() # File should already exist

In [295]:
# Connect to the database
import sqlite3
conn = sqlite3.connect('my_data.db')
c = conn.cursor()

## Creating the Database

Information may be out of date. Current functions for doing this are in `exploring_data.ipynb`


### Dates table

Maps individual dates to schedules in use.

In [3]:
# Create calendar_dates table
c.execute('''CREATE TABLE calendar_dates (id INTEGER PRIMARY KEY, date TEXT, service_id TEXT)''')

# Add "IF NOT EXISTS" after "CREATE TABLE" if we want to modify or append it in the future.
# The types I specified here were ignored. Why?

In [43]:
# Load CSV data into the database
import pandas as pd

# Load the CSV file into a dataframe
dates_df = pd.read_csv('calendar_dates.txt')
fields_to_keep = ['date', 'service_id']
dates_df = dates_df[fields_to_keep]
# Write the data to a sqlite table
dates_df.to_sql('calendar_dates', conn, if_exists='replace', index_label='id')

# if_exists may be fail, replace, or append
# index=False means don't write dataframe index column to the database table. Default is True
# Do we need an index on a many-to-many table?

275

In [296]:
# Check data in dataframe
dates_df.head()

NameError: name 'dates_df' is not defined

In [297]:
# Check data in database

c.execute('''SELECT * FROM calendar_dates LIMIT 5''').fetchall()

[(0, 13105200, 20230108),
 (1, 13105300, 20230109),
 (2, 13105304, 20230109),
 (3, 13105300, 20230110),
 (4, 13105304, 20230110)]

In [299]:
# Read service_ids from database

sunday = c.execute('''SELECT "service_id" FROM "calendar_dates" WHERE "date" = "20230108"''').fetchall()
tuesday = c.execute('''SELECT "service_id" FROM "calendar_dates" WHERE "date" = "20230110"''').fetchall()

# fetchall returns a list of tuples
# Sunday returns a list of one tuple, Tuesday returns a list of two tuples
# The data we need is from the first item in each tuple of the list
# For example, we need to read "tuesday[0][0]" and tuesday[1][0]" to get the service_ids for Tuesday
# These values are of type integer, even though the column is of type text
print (sunday)
print (tuesday)


[(13105200,)]
[(13105300,), (13105304,)]


### Routes table

Add table to show friendly route names to user

In [ ]:
# Create routes table
c.execute('''CREATE TABLE routes (route_id INTEGER PRIMARY KEY, route_short_name TEXT, route_long_name TEXT)''')

In [40]:
# Load the CSV file into a dataframe
routes = pd.read_csv('routes.csv')
fields_to_keep = ['route_id', 'route_short_name', 'route_long_name']
routes = routes[fields_to_keep]
# Write the data to a sqlite table
routes.to_sql('routes', conn, if_exists='replace', index=False)

28

In [41]:
# Test dataframe
routes.head()

,route_id,route_short_name,route_long_name
0,01,01,Campbell Center
1,103,EmX,EmX
2,11,11,Thurston
3,12,12,Gateway
4,13,13,Centennial


In [42]:
# Check data in database

c.execute('''SELECT * FROM routes LIMIT 5''').fetchall()

# We get back an array of tuples, each tuple is a row in the table

[('01', '01', 'Campbell Center'),
 ('103', 'EmX', 'EmX'),
 ('11', '11', 'Thurston'),
 ('12', '12', 'Gateway'),
 ('13', '13', 'Centennial')]

### Trips table

Lists all trips for each route on under each service_id (schedule)

5,240 entries

In [ ]:
# Create trips table
# This seems to be done automatically by pandas
# c.execute('''CREATE TABLE trips (route_id TEXT, service_id TEXT, trip_id TEXT, trip_headsign TEXT, direction_id INTEGER)''')

In [77]:
# Load the CSV file into a dataframe
trips = pd.read_csv('trips.txt')
fields_to_keep = ['trip_id', 'route_id', 'service_id', 'trip_headsign', 'direction_id']
trips = trips[fields_to_keep]
# Write the data to a sqlite table
trips.to_sql('trips', conn, if_exists='replace', index=False, index_label='trip_id')

5240

In [78]:
# Test dataframe
trips.head()

,trip_id,route_id,service_id,trip_headsign,direction_id
0,698920,01,13105100,01 CAMPBELL CENTER,0
1,698921,01,13105100,01 EUGENE STATION,1
2,698944,01,13105100,01 CAMPBELL CENTER,0
3,698945,01,13105100,01 EUGENE STATION,1
4,698962,01,13105100,01 CAMPBELL CENTER,0


In [79]:
# Check data in database
c.execute('''SELECT * FROM trips LIMIT 5''').fetchall()

[(698920, '01', 13105100, '01 CAMPBELL CENTER', 0),
 (698921, '01', 13105100, '01 EUGENE STATION', 1),
 (698944, '01', 13105100, '01 CAMPBELL CENTER', 0),
 (698945, '01', 13105100, '01 EUGENE STATION', 1),
 (698962, '01', 13105100, '01 CAMPBELL CENTER', 0)]

In [83]:
# Make a useful query from the database
# Retrieve all trips for a given route_id and service_id

# Get the service_id for Sunday, 20230528
sunday = c.execute('''SELECT "service_id" FROM "calendar_dates" WHERE "date" = "20230528"''').fetchall()
sunday_svc = sunday[0][0]
print (sunday_svc)

# Get all trips for route_id 103 where service_id=sunday_svc
emx_sun_trips = c.execute('''SELECT "trip_id" FROM "trips" WHERE "route_id" = "103" AND "service_id" = ?''', (sunday_svc,)).fetchall()
# emx_sun_trips_2 = c.execute('''SELECT "trip_id" FROM "trips" WHERE "route_id" = "103" AND "service_id" = "13205200" LIMIT 5''').fetchall()

# print (emx_sun_trips)
print (len(emx_sun_trips))
print (type(emx_sun_trips))

13205200
108
<class 'list'>


In [84]:
triplist = list(map(lambda x:x[0], emx_sun_trips))
print (len(triplist))
print (type(triplist))

108
<class 'list'>


### Stop times file

Lists every time for every stop on every schedule. It's huge.

Eliminating untimed stops reduces it to 48,235 entries.

In [74]:
# Load the CSV file into a dataframe
stop_times = pd.read_csv('stop_times.txt')
# Remove rows with empty departure_time
stop_times = stop_times[stop_times["timepoint"] == 1]
fields_to_keep = ['trip_id', 'departure_time', 'stop_id']
stop_times = stop_times[fields_to_keep]
# Write the data to a sqlite table
stop_times.to_sql('stop_times', conn, if_exists='replace')

48235

In [75]:
# Test dataframe
stop_times.head()

,trip_id,departure_time,stop_id
0,698915,06:31:00,09011
1,698915,06:37:00,09240
2,698915,06:49:00,09236
4,698915,06:56:00,01806
6,698915,07:06:00,02305


In [76]:
# Test database table
c.execute('''SELECT * FROM stop_times LIMIT 5''').fetchall()

[(0, 698915, '06:31:00', '09011'),
 (1, 698915, '06:37:00', '09240'),
 (2, 698915, '06:49:00', '09236'),
 (4, 698915, '06:56:00', '01806'),
 (6, 698915, '07:06:00', '02305')]

In [223]:
print(type(triplist))
print(len(triplist))
print(type(triplist[0]))

<class 'list'>
108
<class 'int'>


In [219]:
# Create a useful query from the database
# Retrieve all departure times for a given stop_id and all trips in triplist

my_stop_id = "02507"
my_trips = tuple(triplist)
print (len(my_trips), " buses leave from station", my_stop_id)
departure_times = c.execute('''
  SELECT "departure_time"
  FROM "stop_times"
  WHERE "stop_id" = ?
  AND "trip_id" IN {}
  ORDER BY "departure_time"'''.format(my_trips),
  (my_stop_id,)).fetchall()
print (len(departure_times), "of those buses are scheduled for today")
print (type(departure_times))

108  buses leave from station 02507
52 of those buses are scheduled for today
<class 'list'>


In [220]:
departure_times = list(map(lambda x:x[0], departure_times))
print (departure_times)

['08:10:00', '08:25:00', '08:40:00', '08:55:00', '09:08:00', '09:24:00', '09:39:00', '09:55:00', '10:10:00', '10:25:00', '10:40:00', '10:55:00', '11:10:00', '11:25:00', '11:40:00', '11:56:00', '12:11:00', '12:26:00', '12:41:00', '12:56:00', '13:11:00', '13:26:00', '13:41:00', '13:56:00', '14:11:00', '14:26:00', '14:41:00', '14:56:00', '15:11:00', '15:26:00', '15:41:00', '15:56:00', '16:11:00', '16:26:00', '16:41:00', '16:56:00', '17:11:00', '17:26:00', '17:41:00', '17:56:00', '18:11:00', '18:26:00', '18:41:00', '18:55:00', '19:10:00', '19:25:00', '19:40:00', '19:55:00', '20:09:00', '20:24:00', '20:39:00', '20:54:00']


## Reading the data

In [390]:
# Get a list of departure times for a given stop_id, route_id, and service_id
def get_dep_times(stop_id, route_id, service_id):
    # Get all trips for route_id and service_id
    trips = c.execute('''SELECT "trip_id" FROM "trips" WHERE "route_id" = ? AND "service_id" = ?''', (route_id, service_id)).fetchall()
    # Convert list of tuples to list of strings
    trips = list(map(lambda x:x[0], trips))
    # Get all departure times for stop_id and trips
    departure_times = c.execute('''
      SELECT "departure_time"
      FROM "stop_times"
      WHERE "stop_id" = ?
      AND "trip_id" IN {}
      ORDER BY "stop_times"'''.format(tuple(trips)),
      (stop_id,)).fetchall()
    # Convert list of tuples to list of strings
    departure_times = list(map(lambda x:x[0], departure_times))
    # Sort the departure times alphabetically
    departure_times.sort()
    return departure_times

In [391]:
spfld_sat_times = get_dep_times("02507", "103", "13205100")
print (len(spfld_sat_times), "Saturday trips")
spfld_sun_times = get_dep_times("02507", "103", "13205200")
print (len(spfld_sun_times), "Sunday trips")
spfld_wkdy_times = get_dep_times("02507", "103", "13205300")
print (len(spfld_wkdy_times), "weekday trips")

64 Saturday trips
52 Sunday trips
75 weekday trips


In [371]:
print(spfld_wkdy_times)
# missing 13:35, 14:50, 15:05, 15:20, ... 17:34?
# Beginning and end of day are OK. Some routes are shorter in
# the middle of the day and aren't counted.

['06:12:00', '06:21:00', '06:41:00', '06:55:00', '07:06:00', '07:12:00', '07:26:00', '07:37:00', '07:44:00', '07:58:00', '08:08:00', '08:18:00', '08:28:00', '08:38:00', '08:48:00', '08:58:00', '09:07:00', '09:17:00', '09:27:00', '09:36:00', '09:47:00', '09:56:00', '10:07:00', '10:16:00', '10:27:00', '10:41:00', '10:57:00', '11:11:00', '11:27:00', '11:41:00', '11:57:00', '12:11:00', '12:27:00', '12:41:00', '12:57:00', '13:12:00', '13:28:00', '13:43:00', '13:58:00', '14:13:00', '14:28:00', '14:43:00', '14:58:00', '15:13:00', '15:28:00', '15:43:00', '15:57:00', '16:12:00', '16:27:00', '16:42:00', '16:57:00', '17:12:00', '17:27:00', '17:42:00', '17:58:00', '18:13:00', '18:27:00', '18:42:00', '18:57:00', '19:11:00', '19:25:00', '19:40:00', '19:55:00', '20:09:00', '20:25:00', '20:39:00', '20:55:00', '21:09:00', '21:25:00', '21:39:00', '21:55:00', '22:09:00', '22:25:00', '22:54:00', '23:24:00']


### Add a function to display time until next bus

In [372]:
# Get time remaining until next departure
import datetime
def get_time_remaining(departure_times):
    # Get current time
    now = datetime.datetime.now()
    # Convert current time to seconds since midnight
    now_seconds = now.hour*3600 + now.minute*60 + now.second
    # Convert departure times to seconds since midnight
    departure_seconds = list(map(lambda x:int(x[0:2])*3600 + int(x[3:5])*60 + int(x[6:8]), departure_times))
    # Find the first departure time that is greater than now_seconds
    for departure in departure_seconds:
        if departure > now_seconds:
            break
    # Calculate time remaining in seconds
    time_remaining = departure - now_seconds
    # Convert time remaining to minutes and seconds
    minutes_remaining = time_remaining // 60
    seconds_remaining = time_remaining % 60
    # Return time remaining as a string
    return str(minutes_remaining) + ":" + str(seconds_remaining).zfill(2)

In [373]:
# Test the time remaining function
get_time_remaining(spfld_wkdy_times)

'3:09'

In [363]:
# Function to get all remaining departure times
def get_all_times_remaining(departure_times):
    # Get current time
    now = datetime.datetime.now()
    # Convert current time to seconds since midnight
    now_seconds = now.hour*3600 + now.minute*60 + now.second
    # Convert departure times to seconds since midnight
    departure_seconds = list(map(lambda x:int(x[0:2])*3600 + int(x[3:5])*60 + int(x[6:8]), departure_times))
    # find the index of the first departure time that is greater than now_seconds
    for i, departure in enumerate(departure_seconds):
        if departure > now_seconds:
            break
    # Get all departure times that are greater than now_seconds
    # The index should be the same for both lists
    remaining_departures = departure_times[i:]
    return remaining_departures


In [374]:
get_all_times_remaining(spfld_wkdy_times)

['11:41:00',
 '11:57:00',
 '12:11:00',
 '12:27:00',
 '12:41:00',
 '12:57:00',
 '13:12:00',
 '13:28:00',
 '13:43:00',
 '13:58:00',
 '14:13:00',
 '14:28:00',
 '14:43:00',
 '14:58:00',
 '15:13:00',
 '15:28:00',
 '15:43:00',
 '15:57:00',
 '16:12:00',
 '16:27:00',
 '16:42:00',
 '16:57:00',
 '17:12:00',
 '17:27:00',
 '17:42:00',
 '17:58:00',
 '18:13:00',
 '18:27:00',
 '18:42:00',
 '18:57:00',
 '19:11:00',
 '19:25:00',
 '19:40:00',
 '19:55:00',
 '20:09:00',
 '20:25:00',
 '20:39:00',
 '20:55:00',
 '21:09:00',
 '21:25:00',
 '21:39:00',
 '21:55:00',
 '22:09:00',
 '22:25:00',
 '22:54:00',
 '23:24:00']

In [375]:
def get_bus_status(departure_times):
    # Get current time
    now = datetime.datetime.now()
    # Convert current time to seconds since midnight
    now_seconds = now.hour*3600 + now.minute*60 + now.second
    # Convert departure times to seconds since midnight
    departure_seconds = list(map(lambda x:int(x[0:2])*3600 + int(x[3:5])*60 + int(x[6:8]), departure_times))
    # Find the first departure time that is greater than now_seconds
    for departure in departure_seconds:
        if departure > now_seconds:
            break
    # Calculate time remaining in seconds
    time_remaining = departure - now_seconds
    # Convert time remaining to minutes and seconds
    minutes_remaining = time_remaining // 60
    seconds_remaining = time_remaining % 60
    # Return time remaining as a string
    if minutes_remaining < 1:
        return "DUE"
    else:
        return str(minutes_remaining) + ":" + str(seconds_remaining).zfill(2)

In [376]:
# Test the bus status function
get_bus_status(spfld_wkdy_times)

'2:28'

In [126]:
# Testing list slicing
my_list = [1,2,3,4,5]
# Get first 3 elements
print(my_list[:3])
# Get last element
print(my_list[-1])
# first next 7 elements
print(my_list[:7])

[1, 2, 3]
5
[1, 2, 3, 4, 5]


Some utility functions to use when calculating times. Use these in the functions above, as well.

In [377]:
# Convert a time in the format of "hh:mm:ss" to seconds past midnight
def tod_to_seconds(time_of_day):
    return int(time_of_day[0:2])*3600 + int(time_of_day[3:5])*60 + int(time_of_day[6:8])

# Convert current time to seconds past midnight
def now_to_seconds():
    # Get current time
    now = datetime.datetime.now()
    # Convert current time to seconds since midnight
    return now.hour*3600 + now.minute*60 + now.second

# Find remaining (h):mm:ss until a time in the format of "hh:mm:ss"
def tod_to_time_remaining(time_of_day, overflow_to_hours=True):
    tod_seconds = tod_to_seconds(time_of_day)
    now_seconds = now_to_seconds()
    seconds_remaining = tod_seconds - now_seconds
    # Convert time remaining to minutes and seconds
    minutes_remaining = seconds_remaining // 60
    seconds_remaining = str(seconds_remaining % 60).zfill(2)
    if minutes_remaining < 1:
        return "DUE"
    # If overflow_to_hours is True, return hours:minutes:seconds
    hours_remaining = ""
    if overflow_to_hours == True:
        hours_remaining = minutes_remaining // 60
        minutes_remaining = str(minutes_remaining % 60).zfill(2)
        if hours_remaining > 0:
            hours_remaining = str(hours_remaining) + ":"
        else:
            hours_remaining = ""
    else: minutes_remaining = str(minutes_remaining)
    # Return time remaining as a string
    # print (type(hours_remaining), type(minutes_remaining), type(seconds_remaining))
    return hours_remaining + minutes_remaining + ":" + seconds_remaining
    

In [378]:
tod_to_time_remaining("23:12:00")

'11:33:12'

In [351]:
# Test the time conversion function
# 12:00:00 should be 43200 seconds past midnight
# 23:59:59 should be 86399 seconds past midnight
# 00:00:00 should be 0 seconds past midnight
# 00:00:01 should be 1 second past midnight
# 24:00:01 should be 86401 seconds past midnight
print(tod_to_seconds("12:00:00"))
print(tod_to_seconds("23:59:59"))
print(tod_to_seconds("00:00:00"))
print(tod_to_seconds("00:00:01"))
print(tod_to_seconds("24:00:01"))

43200
86399
0
1
86401


In [379]:
def get_departure_info(departure_times):
    next_departures = departure_times[0:3]
    last_departure = departure_times[-1]
    next_diffs = [tod_to_time_remaining(x) for x in next_departures]
    last_diff = tod_to_time_remaining(last_departure)

    print(f"Next {len(next_departures)} departures:")
    for i, departure in enumerate(next_departures):
        print(departure + " (" + next_diffs[i] + ")")
    print("Last departure:")
    print(last_departure + " (" + last_diff + ")")


In [380]:
# Test the departure info function
get_departure_info(get_all_times_remaining(spfld_sat_times))

Next 3 departures:
11:40:00 (DUE)
11:56:00 (16:14)
12:11:00 (31:14)
Last departure:
23:23:00 (11:43:14)


In [381]:
# Get all service_ids for a given date
def get_service_ids(date):
    service_ids = c.execute("SELECT service_id FROM calendar_dates WHERE date = ?", (date,)).fetchall()
    # Convert list of tuples to list of strings
    return list(map(lambda x:x[0], service_ids))

In [353]:
schedules_in_use = get_service_ids("20230605")
# Convert list of tuples to list of strings
print(schedules_in_use)

[13205300, 13205304]


In [382]:
# Get all departure times for a stop on a given date
def get_departure_times(route_id, stop_id, date):
    # Get all service_ids for the given date
    service_ids = get_service_ids(date)
    print ("Found", len(service_ids), "service_ids for", date)
    # Get all departure times
    stop_times = c.execute("SELECT departure_time FROM stop_times WHERE stop_id = ? AND service_id IN ({seq})".format(seq=','.join(['?']*len(service_ids))), [stop_id] + service_ids).fetchall()
    # Convert list of tuples to list of strings
    stop_times = list(map(lambda x:x[0], stop_times))
    # Sort the stop_times
    stop_times.sort()
    return stop_times

In [384]:
print(get_departure_times("103", "02507", 20230612))
# print(get_departure_times("91", "02107", 20230610))

Found 2 service_ids for 20230612
['06:12:00', '06:21:00', '06:41:00', '06:55:00', '07:06:00', '07:12:00', '07:26:00', '07:37:00', '07:44:00', '07:58:00', '08:08:00', '08:18:00', '08:28:00', '08:38:00', '08:48:00', '08:58:00', '09:07:00', '09:17:00', '09:27:00', '09:36:00', '09:47:00', '09:56:00', '10:07:00', '10:16:00', '10:27:00', '10:41:00', '10:57:00', '11:11:00', '11:27:00', '11:41:00', '11:57:00', '12:11:00', '12:27:00', '12:41:00', '12:57:00', '13:12:00', '13:28:00', '13:43:00', '13:58:00', '14:13:00', '14:28:00', '14:43:00', '14:58:00', '15:13:00', '15:28:00', '15:43:00', '15:57:00', '16:12:00', '16:27:00', '16:42:00', '16:57:00', '17:12:00', '17:27:00', '17:42:00', '17:58:00', '18:13:00', '18:27:00', '18:42:00', '18:57:00', '19:11:00', '19:25:00', '19:40:00', '19:55:00', '20:09:00', '20:25:00', '20:39:00', '20:55:00', '21:09:00', '21:25:00', '21:39:00', '21:55:00', '22:09:00', '22:25:00', '22:54:00', '23:24:00']


In [385]:
def show_display_board(route_id, stop_id, date):
    all_departure_times = get_departure_times(route_id, stop_id, date)
    if (all_departure_times == []):
        print("No departures found for", route_id, "at", stop_id, "on", date)
        return
    remaining_departure_times = get_all_times_remaining(all_departure_times)
    next_departures = remaining_departure_times[0:3]
    last_departure = remaining_departure_times[-1]
    next_diffs = [tod_to_time_remaining(x) for x in next_departures]
    last_diff = tod_to_time_remaining(last_departure)

    print(f"Next {len(next_departures)} departures:")
    for i, departure in enumerate(next_departures):
        print(departure + " (" + next_diffs[i] + ")")
    print("Last departure:")
    print(last_departure + " (" + last_diff + ")")

In [388]:
spfld_to_eug_stop_id = "02507" # Bay G
eug_to_spfld_stop_id = "02121" # Bay U
eug_to_242_stop_id = "02107"  # Bay G
ranger_station_stop_id = "09083"
agate_inbound_stop_id = "09965"
agate_outbound_stop_id = "09904"

show_display_board(103, spfld_to_eug_stop_id, 20230608)
# show_display_board(103, eug_to_spfld_stop_id, 20230608)
# show_display_board(91, eug_to_242_stop_id, 20230610)
# show_display_board(91, ranger_station_stop_id, 20230611)


Found 2 service_ids for 20230608
Next 3 departures:
11:57:00 (04:41)
12:11:00 (18:41)
12:27:00 (34:41)
Last departure:
23:24:00 (11:31:41)


### Close the database

Clean up loose ends when we finish

In [36]:
conn.close()


---

Test a connection to Postgres instead

In [275]:
# Connect to the database
# import psycopg2

# conn = psycopg2.connect(database="ltd",
#                         host="localhost",
#                         user="postgres",
#                         password="",
#                         port="5432")

In [284]:
# Find all stops on route 91
# c = conn.cursor()
stops_on_91 = c.execute("SELECT DISTINCT stop_id FROM stop_times WHERE trip_id IN (SELECT trip_id FROM trips WHERE route_id = '91')").fetchall()
# Convert list of tuples to list of strings
stops_on_91 = list(map(lambda x:x[0], stops_on_91))
print(stops_on_91)
# Get all stop names for route 91
stop_names_on_91 = c.execute("SELECT DISTINCT stop_name FROM stops WHERE stop_id IN ({seq})".format(seq=','.join(['?']*len(stops_on_91))), stops_on_91).fetchall()
print(stop_names_on_91)

['02107', '01776', '09171', '09026', '09035', '09047', '09055', '09061', '09065', '09077', '09083', '09078', '09066', '09062', '09056', '09048', '09036', '09001', '09021', '00019', 'escenter', '01067', '02501']
[('N/S of Main W of 58th',), ('S/S of Franklin Blvd E of Onyx',), ('Thurston Sta, Bay B (outb.)',), ('Eugene Station, Bay G',), ('Springfield Station, Bay A',), ('N/S of Hwy 126 (MP 12.7) E of Thriftway',), ('N/S of Hwy 126 (MP 9.7) E of Thurston Rd',), ('S/S of Hwy 126 (MP 12.7) E of Walterville',), ('S/S of Hwy 126 (MP 20.1) E of Mailbox 44895',), ('N/S of Hwy 126 (MP 20.0) E of Johnson Crk',), ('S/S of Hwy 126 (MP 26.4) W of Gate Creek Rd',), ('N/S of Hwy 126 (MP 26.4) E of Gate Creek Rd',), ('S/S of Hwy 126 (MP 34.5) W of Nimrod',), ('N/S of Hwy 126 (MP 34.6) E of Nimrod',), ('S/S of Hwy 126 (MP 38.0) W of Finn Rock Store',), ('N/S of Hwy 126 (MP 38.2) E of Finn Rock Store',), ('S/S of Blue Riv Dr E of McKenzie St',), ('N/S of Blue River Dr W of Dexter St',), ('S/S of Hwy126

In [301]:
c.execute("DROP TABLE IF EXISTS dates")